In [19]:
!pip install --force-reinstall numpy==1.26.4
# Ctrl + M -> . after running this to restart session

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import numpy as np
import torch

print("NumPy version:", np.__version__)
print("PyTorch version:", torch.__version__)


NumPy version: 1.26.4
PyTorch version: 2.6.0+cu124


In [2]:
!pip install transformers datasets sklearn
!pip install datasets transformers
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
os.environ["WANDB_DISABLED"] = "true"
from google.colab import drive
drive.mount('/content/drive')

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_parquet("drive/MyDrive/nyt_data.parquet")

# Drop rows with missing or empty excerpts
df = df.dropna(subset=["excerpt"])
df = df[df["excerpt"].str.strip() != ""]

# Convert year to decade
df["decade"] = (df["year"] // 10) * 10

# Sample up to 10,000 rows per decade
balanced_df = df.groupby("decade", group_keys=False).apply(
    lambda g: g.sample(n=min(2000, len(g)), random_state=42)
)

# Save the filtered + balanced dataset
balanced_df.to_parquet("nyt_1k_per_decade.parquet")

<ipython-input-18-cfec9a8c775d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["decade"] = (df["year"] // 10) * 10
<ipython-input-18-cfec9a8c775d>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df.groupby("decade", group_keys=False).apply(


In [4]:
# Use already-processed 'decade' column and create label_map
def preprocess_clean_parquet(parquet_path):
    df = pd.read_parquet(parquet_path)

    # Use existing 'decade' column and build label_map directly
    label_names = sorted(df["decade"].unique())
    label_map = {str(name): idx for idx, name in enumerate(label_names)}

    df["label"] = df["decade"].astype(str).map(label_map)
    return df, label_map

def split_and_chunk(df, tokenizer, label_map, max_length=512, chunks_per_item=2):
    texts, labels = [], []
    for _, row in df.iterrows():
        content = row["excerpt"]
        label = row["label"]

        try:
            tokens = tokenizer(content, truncation=False)["input_ids"]
            chunks = [
                tokens[i:i+max_length]
                for i in range(0, len(tokens), max_length)
                if len(tokens[i:i+max_length]) >= 10
            ]
            random.shuffle(chunks)
            selected = chunks[:chunks_per_item]
            for chunk in selected:
                texts.append(tokenizer.decode(chunk))
                labels.append(label)
        except Exception as e:
            print(f"Tokenization error: {e}")

    return texts, labels

# --- Use updated preprocessing ---

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
#parquet_path = "drive/MyDrive/nyt_data.parquet"
parquet_path = "drive/MyDrive/nyt_2k_per_decade.parquet"

# Step 1: Load already-cleaned dataset
df, label_map = preprocess_clean_parquet(parquet_path)

# Step 2: Train/Val/Test split
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Step 3: Chunk excerpts and tokenize
train_texts, train_labels = split_and_chunk(train_df, tokenizer, label_map)
val_texts, val_labels = split_and_chunk(val_df, tokenizer, label_map)
test_texts, test_labels = split_and_chunk(test_df, tokenizer, label_map)

# Step 4: Create HuggingFace datasets
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})


Token indices sequence length is longer than the specified maximum sequence length for this model (1623 > 512). Running this sequence through the model will result in indexing errors


In [5]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Apply to all datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/12856 [00:00<?, ? examples/s]

Map:   0%|          | 0/2745 [00:00<?, ? examples/s]

Map:   0%|          | 0/2751 [00:00<?, ? examples/s]

In [6]:
# load BERT's pretrained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))

# training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    save_strategy="no",
    weight_decay=0.01,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,1.336700,1.095667


TrainOutput(global_step=3214, training_loss=1.3366907353570316, metrics={'train_runtime': 1326.5151, 'train_samples_per_second': 9.692, 'train_steps_per_second': 2.423, 'total_flos': 3382829063135232.0, 'train_loss': 1.3366907353570316, 'epoch': 1.0})

In [7]:
model.save_pretrained("/content/drive/MyDrive/BERT_saved_model_NYT_2k/")
tokenizer.save_pretrained("/content/drive/MyDrive/BERT_saved_model_NYT_2k/")

# v3 is train-test split before splitting each book into smaller sets

('/content/drive/MyDrive/BERT_saved_model_NYT_2k/tokenizer_config.json',
 '/content/drive/MyDrive/BERT_saved_model_NYT_2k/special_tokens_map.json',
 '/content/drive/MyDrive/BERT_saved_model_NYT_2k/vocab.txt',
 '/content/drive/MyDrive/BERT_saved_model_NYT_2k/added_tokens.json',
 '/content/drive/MyDrive/BERT_saved_model_NYT_2k/tokenizer.json')

In [ ]:
# load saved model instead of training again

model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/BERT_saved_modelv3/")
tokenizer = BertTokenizerFast.from_pretrained("/content/drive/MyDrive/BERT_saved_modelv3/")


# dummy training args just for prediction — no training will happen
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
)


In [8]:
from sklearn.metrics import accuracy_score
# get predictions
preds_output = trainer.predict(test_dataset)

# extract predictions and true labels
preds = preds_output.predictions.argmax(axis=1)
labels = preds_output.label_ids

# accuracy
accuracy = accuracy_score(labels, preds)
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5325


In [9]:
correct = np.abs(preds - labels) <= 1
accuracy = correct.sum() / len(labels)

print(f"Validation Accuracy (±1 tolerance): {accuracy:.4f}")


Validation Accuracy (±1 tolerance): 0.8564


In [10]:
# Pred and labels are NumPy arrays
average_diff = np.mean(np.abs(preds - labels))
print(f"Average Difference: {average_diff:.4f}")
#NYT 2k

Average Difference: 0.7299
